In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE


In [2]:
# Function to Load Data
def load_data_from_directory(directory_path):
    data_frames = []
    for file_name in os.listdir(directory_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory_path, file_name)
            df = pd.read_csv(file_path)
            data_frames.append(df)
    return pd.concat(data_frames, ignore_index=True)




In [3]:
# Function to Preprocess Data
# Convert hexadecimal strings to integers
def hex_to_int(x):
    try:
        return int(str(x), 16)
    except ValueError:
        return 0  # Handle non-hexadecimal values
        
def preprocess_data(df):
    df['timestamp'] = pd.to_datetime(df['timestamp']).astype(np.int64) // 10**9
    df['arbitration_id'] = df['arbitration_id'].apply(hex_to_int)
    df['data_field'] = df['data_field'].apply(hex_to_int)
    return df

# Function to Extract Features and Labels
def extract_features_labels(df):
    X = df.drop(columns='attack')
    y = df['attack']
    return X, y

In [4]:
# Function to Train Isolation Forest Model
def train_isolation_forest(X_train, y_train):
    # Apply SMOTE for Handling Class Imbalance
    smote = SMOTE(random_state=42)
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

    # Standardize the Features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_resampled)

    # Train the Isolation Forest Model
    iso_forest = IsolationForest(contamination=0.1, random_state=42)

    # Record training start time
    start_time = time.time()
    
    iso_forest.fit(X_train_scaled)

    # Calculate training time
    training_time = time.time() - start_time
    
    return iso_forest, scaler, training_time

# Function to Test Isolation Forest Model
def test_isolation_forest(iso_forest, scaler, X_test, y_test):
    # Standardize the Features
    X_test_scaled = scaler.transform(X_test)

    # Record testing start time
    start_time = time.time()
    # Predict Outliers on Test Data
    test_predictions = iso_forest.predict(X_test_scaled)

    # Calculate testing time
    testing_time = time.time() - start_time
    
    test_labels = np.where(test_predictions == -1, 1, 0)  # Convert -1 to 1 (attack) and 1 to 0 (normal)

    # Evaluate Test Performance
    conf_matrix_test = confusion_matrix(y_test, test_labels)
    mcc_test = matthews_corrcoef(y_test, test_labels)
    accuracy_test = accuracy_score(y_test, test_labels)
    precision_test = precision_score(y_test, test_labels, average='weighted', zero_division=0)
    recall_test = recall_score(y_test, test_labels, average='weighted', zero_division=0)
    f1_test = f1_score(y_test, test_labels, average='weighted', zero_division=0)
    informedness_test = recall_test - (1 - recall_test)
    markedness_test = precision_test - (1 - precision_test)

    return {
        'conf_matrix': conf_matrix_test,
        'mcc': mcc_test,
        'accuracy': accuracy_test,
        'precision': precision_test,
        'recall': recall_test,
        'f1_score': f1_test,
        'informedness': informedness_test,
        'markedness': markedness_test,
        'testing_time': testing_time
    }


In [5]:
# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_01/train_01/'
]

test_directories = [
    './Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_01/test_03_known_vehicle_unknown_attack/',
    './Documents/Research/can-train-and-test/set_01/test_04_unknown_vehicle_unknown_attack/'
]

# Iterate through each training directory
for train_dir in train_directories:
    
    # Load and preprocess training data
    df_train = load_data_from_directory(train_dir)
    df_train = preprocess_data(df_train)
    X_train, y_train = extract_features_labels(df_train)

    # Train the Isolation Forest Model
    iso_forest, scaler, training_time = train_isolation_forest(X_train, y_train)

    # Corresponding test directories for each training set
    corresponding_tests = [td for td in test_directories if td.startswith(train_dir.split('/')[0])]
    
    # Test the model on each corresponding test set
    for test_dir in corresponding_tests:
        
        # Load and preprocess testing data
        df_test = load_data_from_directory(test_dir)
        df_test = preprocess_data(df_test)
        X_test, y_test = extract_features_labels(df_test)

        # Test the Isolation Forest Model
        test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

        # Print Testing Metrics
        print(f"\n===============================================================================")
        print(f"\nTesting Metrics (Train: {train_dir} | Test: {test_dir}):")
        print(f"\n-----------------------------------------------------------------------")
        print(f"\nTraining Time: {training_time:.2f} seconds")
        print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
        print(f"\n-----------------------------------------------------------------------")
        print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
        print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
        print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
        print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
        print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
        print(f"Informedness: {test_metrics['informedness']:.4f}")
        print(f"Markedness: {test_metrics['markedness']:.4f}")




Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_01/train_01/ | Test: ./Documents/Research/can-train-and-test/set_01/test_01_known_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 403.07 seconds
Testing Time: 116.85 seconds

-----------------------------------------------------------------------
Accuracy: 79.78%
Precision: 98.42%
Recall: 79.78%
F1-Score: 87.75%
Matthews Correlation Coefficient: 0.1123
Informedness: 0.5956
Markedness: 0.9684


Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_01/train_01/ | Test: ./Documents/Research/can-train-and-test/set_01/test_02_unknown_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 403.07 seconds
Testing Time: 121.63 seconds

-----------------------------------------------------------------------
Accuracy: 64.93%
Precision: 97.47%
Recall: 64.93%
F1-Score: 76.41%
Matthews Correlatio

In [6]:
# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_02/train_01/'
]

test_directories = [
    './Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_02/test_03_known_vehicle_unknown_attack/',
    './Documents/Research/can-train-and-test/set_02/test_04_unknown_vehicle_unknown_attack/'
]

# Iterate through each training directory
for train_dir in train_directories:
    
    # Load and preprocess training data
    df_train = load_data_from_directory(train_dir)
    df_train = preprocess_data(df_train)
    X_train, y_train = extract_features_labels(df_train)

    # Train the Isolation Forest Model
    iso_forest, scaler, training_time = train_isolation_forest(X_train, y_train)

    # Corresponding test directories for each training set
    corresponding_tests = [td for td in test_directories if td.startswith(train_dir.split('/')[0])]
    
    # Test the model on each corresponding test set
    for test_dir in corresponding_tests:
        
        # Load and preprocess testing data
        df_test = load_data_from_directory(test_dir)
        df_test = preprocess_data(df_test)
        X_test, y_test = extract_features_labels(df_test)

        # Test the Isolation Forest Model
        test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

        # Print Testing Metrics
        print(f"\n===============================================================================")
        print(f"\nTesting Metrics (Train: {train_dir} | Test: {test_dir}):")
        print(f"\n-----------------------------------------------------------------------")
        print(f"\nTraining Time: {training_time:.2f} seconds")
        print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
        print(f"\n-----------------------------------------------------------------------")
        print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
        print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
        print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
        print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
        print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
        print(f"Informedness: {test_metrics['informedness']:.4f}")
        print(f"Markedness: {test_metrics['markedness']:.4f}")




Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_02/train_01/ | Test: ./Documents/Research/can-train-and-test/set_02/test_01_known_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 743.70 seconds
Testing Time: 311.68 seconds

-----------------------------------------------------------------------
Accuracy: 99.75%
Precision: 99.81%
Recall: 99.75%
F1-Score: 99.78%
Matthews Correlation Coefficient: 0.1300
Informedness: 0.9949
Markedness: 0.9963


Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_02/train_01/ | Test: ./Documents/Research/can-train-and-test/set_02/test_02_unknown_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 743.70 seconds
Testing Time: 187.99 seconds

-----------------------------------------------------------------------
Accuracy: 80.14%
Precision: 99.54%
Recall: 80.14%
F1-Score: 88.76%
Matthews Correlatio

In [7]:
# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_03/train_01/'
]

test_directories = [
    './Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_03/test_03_known_vehicle_unknown_attack/',
    './Documents/Research/can-train-and-test/set_03/test_04_unknown_vehicle_unknown_attack/'
]

# Iterate through each training directory
for train_dir in train_directories:
    
    # Load and preprocess training data
    df_train = load_data_from_directory(train_dir)
    df_train = preprocess_data(df_train)
    X_train, y_train = extract_features_labels(df_train)

    # Train the Isolation Forest Model
    iso_forest, scaler, training_time = train_isolation_forest(X_train, y_train)

    # Corresponding test directories for each training set
    corresponding_tests = [td for td in test_directories if td.startswith(train_dir.split('/')[0])]
    
    # Test the model on each corresponding test set
    for test_dir in corresponding_tests:
        
        # Load and preprocess testing data
        df_test = load_data_from_directory(test_dir)
        df_test = preprocess_data(df_test)
        X_test, y_test = extract_features_labels(df_test)

        # Test the Isolation Forest Model
        test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

        # Print Testing Metrics
        print(f"\n===============================================================================")
        print(f"\nTesting Metrics (Train: {train_dir} | Test: {test_dir}):")
        print(f"\n-----------------------------------------------------------------------")
        print(f"\nTraining Time: {training_time:.2f} seconds")
        print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
        print(f"\n-----------------------------------------------------------------------")
        print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
        print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
        print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
        print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
        print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
        print(f"Informedness: {test_metrics['informedness']:.4f}")
        print(f"Markedness: {test_metrics['markedness']:.4f}")




Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_03/train_01/ | Test: ./Documents/Research/can-train-and-test/set_03/test_01_known_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 523.84 seconds
Testing Time: 202.38 seconds

-----------------------------------------------------------------------
Accuracy: 80.22%
Precision: 96.39%
Recall: 80.22%
F1-Score: 87.36%
Matthews Correlation Coefficient: 0.0150
Informedness: 0.6044
Markedness: 0.9277


Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_03/train_01/ | Test: ./Documents/Research/can-train-and-test/set_03/test_02_unknown_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 523.84 seconds
Testing Time: 147.15 seconds

-----------------------------------------------------------------------
Accuracy: 43.29%
Precision: 95.02%
Recall: 43.29%
F1-Score: 58.55%
Matthews Correlatio

In [8]:
# Directories
train_directories = [
    './Documents/Research/can-train-and-test/set_04/train_01/'
]

test_directories = [
    './Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/',
    './Documents/Research/can-train-and-test/set_04/test_03_known_vehicle_unknown_attack/',
    './Documents/Research/can-train-and-test/set_04/test_04_unknown_vehicle_unknown_attack/'
]

# Iterate through each training directory
for train_dir in train_directories:
    
    # Load and preprocess training data
    df_train = load_data_from_directory(train_dir)
    df_train = preprocess_data(df_train)
    X_train, y_train = extract_features_labels(df_train)

    # Train the Isolation Forest Model
    iso_forest, scaler, training_time = train_isolation_forest(X_train, y_train)
    
    # Corresponding test directories for each training set
    corresponding_tests = [td for td in test_directories if td.startswith(train_dir.split('/')[0])]
    
    # Test the model on each corresponding test set
    for test_dir in corresponding_tests:
        
        # Load and preprocess testing data
        df_test = load_data_from_directory(test_dir)
        df_test = preprocess_data(df_test)
        X_test, y_test = extract_features_labels(df_test)

        # Test the Isolation Forest Model
        test_metrics = test_isolation_forest(iso_forest, scaler, X_test, y_test)

        # Print Testing Metrics
        print(f"\n===============================================================================")
        print(f"\nTesting Metrics (Train: {train_dir} | Test: {test_dir}):")
        print(f"\n-----------------------------------------------------------------------")
        print(f"\nTraining Time: {training_time:.2f} seconds")
        print(f"Testing Time: {test_metrics['testing_time']:.2f} seconds")
        print(f"\n-----------------------------------------------------------------------")
        print(f"Accuracy: {test_metrics['accuracy'] * 100:.2f}%")
        print(f"Precision: {test_metrics['precision'] * 100:.2f}%")
        print(f"Recall: {test_metrics['recall'] * 100:.2f}%")
        print(f"F1-Score: {test_metrics['f1_score'] * 100:.2f}%")
        print(f"Matthews Correlation Coefficient: {test_metrics['mcc']:.4f}")
        print(f"Informedness: {test_metrics['informedness']:.4f}")
        print(f"Markedness: {test_metrics['markedness']:.4f}")




Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_04/train_01/ | Test: ./Documents/Research/can-train-and-test/set_04/test_01_known_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 476.83 seconds
Testing Time: 165.44 seconds

-----------------------------------------------------------------------
Accuracy: 91.96%
Precision: 95.65%
Recall: 91.96%
F1-Score: 93.76%
Matthews Correlation Coefficient: -0.0197
Informedness: 0.8393
Markedness: 0.9130


Testing Metrics (Train: ./Documents/Research/can-train-and-test/set_04/train_01/ | Test: ./Documents/Research/can-train-and-test/set_04/test_02_unknown_vehicle_known_attack/):

-----------------------------------------------------------------------

Training Time: 476.83 seconds
Testing Time: 407.74 seconds

-----------------------------------------------------------------------
Accuracy: 98.70%
Precision: 99.45%
Recall: 98.70%
F1-Score: 99.06%
Matthews Correlati